In [2]:
import numpy as np
import math
import scipy.special
import pprint

class ComplicatedNeuralNetwork:
    
    def __init__(self, learningrate: float, inputnodes: int, outputnodes: int, hiddennodes_list: list = None):
        '''Create neural network'''
        
        # create list that contains nodes count for every layer
        
        # if None - NN doesnt contains hidden layers
        if hiddennodes_list is None or len(hiddennodes_list) == 0:
            layers_list = [inputnodes, outputnodes]
        else:
            layers_list = [inputnodes] + hiddennodes_list + [outputnodes]
        
        # create weights matrix for every layer
        self._weights = []
        
        # about matrix dimenshion:
        # matrixs count = layers count - 1
        # matrix = [row] x [column] = [next layer] x [previous layer] => range(len(layers_list) - 1)
        
        # create start weights
        for i in range(len(layers_list) - 1):
            self._weights.append(np.random.normal(0.0, math.pow(layers_list[i + 1], -0.5), 
                                                  (layers_list[i + 1], layers_list[i])))
        # learning rate
        self._lr = learningrate
        
        # activation function
        self._act_func = lambda x: scipy.special.expit(x)
    
    def train(self, inputs_list, targets_list):
        
        inputs = np.array(inputs_list, ndmin=2).T
        targets = np.array(targets_list, ndmin=2).T
        
        # COLLECT ALL OUTPUTS
        
        output_column = inputs
        outputs_list = [inputs]
        
        for wm in self._weights:
            output_column = self._act_func(np.dot(wm, output_column))
            outputs_list.append(output_column)
        
        # COLLECT ALL ERRORS
        
        output_errors = targets - output_column
        errors_list = [output_errors]
        
        for wm in self._weights[:0:-1]:
            output_errors = np.dot(wm.T, output_errors)
            errors_list.insert(0, output_errors)
        
        # IMPROVE WEIGHTS IN MATRIX
        
        for i in range(len(self._weights)):
            self._weights[i] += self._lr * np.dot(errors_list[i] * outputs_list[i+1] * (1.0 - outputs_list[i+1]), 
                                                  outputs_list[i].T)      
    
    def query(self, inputs):
        '''Calculate result with inputs values'''
        
        # transform row in column to multiple on matrix
        res_column = np.array(inputs, ndmin=2).T
        
        for wm in self._weights:
            res_column = self._act_func(np.dot(wm, res_column))
        
        return res_column        
    
    def print_layers(self):
        
        for wm in self._weights:
            pprint.pprint(wm)
            
    @property
    def weights(self):
        #
        return self._weights
    
    @weights.setter
    def weights(self, weights):
        #
        self._weights = weights
        

In [3]:
cn = ComplicatedNeuralNetwork(0.5, 2, 2, [1, 2, 3])
print('Layers:')
cn.print_layers()
print()
print('Query result: ', cn.query([0.1, 0.2]))
cn.train([0.1, 0.2], [0.5, 0.5])
print('Layers:')
cn.print_layers()
print()
print('Query result: ', cn.query([0.1, 0.2]))

Layers:
array([[ 1.37088393, -0.14577665]])
array([[-0.52477637],
       [-0.46452451]])
array([[-0.46637727,  0.47940431],
       [ 0.84382451, -0.61402119],
       [ 0.14347606,  0.03665084]])
array([[ 1.1863614 , -1.38075678,  0.43549985],
       [-1.29707726,  1.41867368, -0.13054062]])

Query result:  [[0.52479922]
 [0.50584423]]
Layers:
array([[ 1.37083851, -0.1458675 ]])
array([[-0.52279599],
       [-0.46626185]])
array([[-0.46755473,  0.47820555],
       [ 0.84522048, -0.61259997],
       [ 0.14293575,  0.03610075]])
array([[ 1.18480803, -1.38237578,  0.43389346],
       [-1.29744419,  1.41829126, -0.13092006]])

Query result:  [[0.52394386]
 [0.50594458]]


In [4]:
res = cn.query([0.1, 0.2])
print(res[0], res[1])

[0.52394386] [0.50594458]
